## 使用的資料說明

我在金門開了一間酒吧叫做夢酒館，這一個專案本來是學校作業，當時剛好在寫補助案，想到可以做一個 RAG 的內部資料集，因為常常在申請政府案子，很常需要重複整理有關公司資訊等等的，因此認為可以把相關資訊都全部放進 RAG，未來可以根據不同的專案，提取資料並且快速整合撰寫成不同方向的計劃書。

目前因為是作業實作，先將機器人設定為是對外的品牌介紹人員，分享給朋友體驗的時候會比較有趣。

### 目前裡面的資料有
1. 品牌介紹
2. 調酒酒單介紹
3. 相關媒體報導

### 發現比較大的問題如下:
* 複合式問句： 發現 RAG 系統一次只能處理單一問題，對於包含多個子句的複合式問句，無法精確理解並提供完整答案。
    * 目前解法：為了解決這個問題，利用 LLM 先將複合式問句拆解成多個子問題，分別進行檢索，最後再整合所有答案。

* 內容組合錯誤： 由於 RAG 系統將內容切割成 chunks，在檢索過程中，偶爾會出現內容組合錯誤，例如詢問 A 調酒卻得到 B 調酒的介紹。目前已發現問題，但仍在尋找最佳解法。
    * 目前解法：有發現問題但暫時還沒辦法完全理解和實作出解法，因此還沒有更新到程式內。

## 1. 引入必要套件

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [6]:
from openai import OpenAI
import gradio as gr
import ast
import tqdm as notebook_tqdm

## 2. 自訂 E5 embedding 類別

In [7]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

## 3. 載入 `faiss_db`

In [8]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

## 4. 設定好我們要的 LLM

In [ ]:
import os
import dotenv
from openai import OpenAI

dotenv.load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

model = "gpt-4.1-nano"

client = OpenAI(api_key=api_key)

## 6. 使用 RAG 來回應

為了解決，複合式問句的檢索準確度，重新設計檢索和回答的流程


1. 語意拆解： 使用大型語言模型（LLM）將使用者輸入的複合式問題，拆解成多個語意清晰、可獨立檢索的子問題。
2. 個別檢索： 針對每個拆解後的子問題，分別從知識庫中檢索相關文件。
3. 整合回答： 根據原始問句，以及步驟二中檢索到的所有相關文件，使用 LLM 整合生成最終的完整回覆。

### 6.1 語意拆解
透過 LLM 將 user input 拆解成很多個小子句


In [10]:
# 透過語意拆解對話成很多個小子句

system_prompt_seperate = """
你是一位專門協助語意拆解的 AI 工具設計師，任務是將使用者輸入的複合問題，轉換成清楚、可獨立檢索的子問題。

請依照以下規則進行拆解：

1. 若問題中包含兩個以上的子句（例如多個動作、主詞、或時間條件），請將其拆成多個子問題。
2. 每個子問題應該是完整的句子，能夠獨立被語意檢索模型理解。
3. 拆解後請用 Python list 格式回傳，例如：[夢酒館登過哪些國際媒體，分別是哪些調酒？]，請拆解成
["夢酒館有登過哪些國際媒體？", "夢酒館登上國際媒體的是哪些調酒？"]
4. 若問題本身已經是單一子句，請回傳原句構成的 list。
5. 除了 Python list 之外，不要回傳任何說明。

請注意：不要自行補充或改寫原始問題的語意，只做語意拆解。
"""

input_prompt_seperate = """
請將下列文字，重新拆解成不同的子句，並且整理成一個 Python list 回傳。
{queries}
"""

def seperate_queries(user_input):
    # 將自定 prompt 套入格式
    final_prompt = input_prompt_seperate.format(queries=user_input)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt_seperate},
            {"role": "user", "content": final_prompt},
        ]
    )
    raw_output = response.choices[0].message.content
    sub_queries = ast.literal_eval(raw_output)

    return {
        "main_query": user_input, # 把原始問題也保留
        "sub_queries": sub_queries
    }

# 測試與示範
user_input = "我喜歡清爽氣泡的調酒，請你推薦我一杯調酒。同時也請你跟我說明夢酒館在地方所做的貢獻，還有夢酒館登過的國際媒體報導。"

# 將主要輸入分解成多個子句
result = seperate_queries(user_input)
main_query = result['main_query']
queries_list = result['sub_queries']

# 輸出原始問句和分解後的子句
print(f"原始問句是：\n{result['main_query']}\n")
print("分解後的子問句是：")
for query in queries_list:
    i = queries_list.index(query) + 1
    print(f"子問句{i}：{query}")

原始問句是：
我喜歡清爽氣泡的調酒，請你推薦我一杯調酒。同時也請你跟我說明夢酒館在地方所做的貢獻，還有夢酒館登過的國際媒體報導。

分解後的子問句是：
子問句1：我喜歡清爽氣泡的調酒。
子問句2：請你推薦我一杯調酒。
子問句3：請你跟我說明夢酒館在地方所做的貢獻。
子問句4：請你說明夢酒館登過的國際媒體報導。


### 6.2 個別檢索
分別針對每一個子句檢索資料

In [11]:
#分別檢索取回資料並使用 top k = 3 讓資料更完整
def retrieve_answers(queries_list, top_k=3):
    results = []
    for sub_query in queries_list:
        docs = retriever.invoke(sub_query, config=dict(k=top_k)) # Gemini 說 BaseRetriever.get_relevant_documents 之後不支援，要我改成 invoke
        # 做一個空白的 list 並存入檢索後的資料
        retrieved_chunks = []

        for doc in docs:
            retrieved_chunks.append(doc.page_content)
        results.append({
            "sub_query": sub_query,
            "chunks": retrieved_chunks
        })
    return results

# 測試與示範(延續上面的 user_input)
retrieved_answers = retrieve_answers(queries_list)

# 輸出每一個子句檢索回來的答案
print(f"共有{len(retrieved_answers)}組問答，Q&A分別是:\n")

for answer_pairs in retrieved_answers:
    sub_query = answer_pairs["sub_query"]
    chunks = answer_pairs["chunks"]
    print(f"Sub_query: {sub_query}\n Chunks: {chunks}\n")

共有4組問答，Q&A分別是:

Sub_query: 我喜歡清爽氣泡的調酒。
 Chunks: ['**原料**  \n琴酒（琴酒）、百香果、芳香萬壽菊、茉莉綠茶、蜂蜜、氣泡水\n\n**原料意象**  \n蜂蜜綠茶：清新、甜美的湖邊約會。芳香萬壽菊：香草類植物，湖畔草地的感覺。氣泡調酒：輕盈爽口，輕鬆日常的約會感。\n\n**製作技法**  \n奶洗：原理是牛奶加入酸性物質會凝固，咖啡濾紙過濾掉凝固的東西後就可以得到澄清的液體。我們的做法是，把泡得過濃的蜂蜜綠茶（要澄清但又要茶味夠重所以我們刻意泡超久）還有其他外料都加好，接著加入牛奶，等他凝固後，最後全部過濾完，奶洗的過程剛好會把單寧一起濾掉，得到有茶味但是不苦澀的蜂蜜綠茶。\n\n**風味描述**  \n清爽、淡雅、氣泡感明顯，有如蜂蜜綠茶般的親切甜感，帶有花香與果酸，入口輕盈、毫無負擔。宛如初戀般的清新甜蜜，適合任何時刻來上一杯。\n\n**適合對象**  \n喜歡清爽、無負擔調酒的人；喜愛花果茶系風味、偏好輕酒感者；尋找一杯日常也能喝的低壓力調酒者。\n\n**視覺意象**  \n金黃蜂蜜色、自然、愜意、午後陽光下的草皮約會感。\n\n**隨杯明信片內容內容**  \n天鵝船\n\n\n---', '**適合對象**  \n喜歡清爽、無負擔調酒的人；喜愛花果茶系風味、偏好輕酒感者；尋找一杯日常也能喝的低壓力調酒者。\n\n**視覺意象**  \n金黃蜂蜜色、自然、愜意、午後陽光下的草皮約會感。\n\n**隨杯明信片內容內容**  \n天鵝船\n\n\n---\n\n---\n調酒名稱: "小鳳出逃"\n價格: 360\n酒精濃度: 9.48\ntags: [\'金門調酒\', \'特色酒單\']\n---\n\n**故事**  \n1999年的颱風把畜試所的孔雀園吹爆造成孔雀跑出去繁殖。颱風把畜試所的孔雀園吹爆造成孔雀跑出去繁殖，原先設計時以 PAVAN 藍孔雀葡萄利口酒，和金門野外也都會看到的迷迭香，來表示現在金門野外都會看到的藍孔雀。希臘琴酒，洋乳香（一個特殊的樹脂）特殊氣味和其他材料帶來的木質調性，把迷迭香和葡萄風味連接在一起。（現在因為 PAVAN 台灣斷貨，所以換成台灣產的麝香葡萄利口酒）\n\n**原料**  \n琴酒、麝香葡萄利口酒、迷迭香、氣泡水\n\n**原料意象**  \n氣泡水：象徵颱

### 6.3 整合回答
把6.1 6.2得到的資料，請 LLM 整合並回答。

In [12]:
#使用 user input　當成問句，並使用檢索之後的資料提共給　LLM 當作內容基準，可以避免分割成子句後，失去了原本完整問句的語意和情緒，但又可以透過檢索回來的資料，提升回答準確度。

system_prompt = (
    "你是『把夢酒館介紹給旅客的品牌大使』—— 一位兼具雞尾酒專業、地方文化策展力的品牌介紹人員。"
    "你的任務是依據被提共的內容，親切生動地向來訪旅客講解夢酒館的故事、雞尾酒靈感及我們在金門推動的地方創生行動。"
    "請以繁體中文作為回答語言，回答要溫暖、專業。"
    "請根據來源資料回答，若資料中未明確提及，請不要自行組合資訊。"
)

prompt_template = """
旅客問題：
{main_query}

根據下列資料來回應訪客問題，務必貼合資料細節並保持品牌語氣：
{retrieved_answers}

請依據下列規範作答：
1. 可以的話，附上相關連結或實際案例說明。
2. 如果知識庫無法支援完整答案，請誠實說明並提出可協助的後續方向 (ex. 請 Email 到 contact@mojokm.com)，可以推薦一個最接近的，但不要自行組合資訊。
3. 請一定要全程使用台灣人慣用的繁體中文來回答。
4. 在推薦調酒之前，請先了解對方的口味喜好，否則以品牌故事理念為主要回答。
5. 請避免將不同調酒的描述混合使用，除非資料中有明確比較。
"""

chat_history = []

def integrate_answers(main_query, retrieved_answers):

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(main_query=main_query, retrieved_answers=retrieved_answers)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((main_query, answer))
    return answer

# 測試與示範
print(f"原始問句是:\n\n「{main_query}」\n\n將會使用這句話當作主要的問句給LLM\n\n\n")

# 整合檢索之後的回答
answer = integrate_answers(main_query, retrieved_answers)
print(f"整合之後的回答是:\n\n{answer}")
print('\n\n\n')

原始問句是:

「我喜歡清爽氣泡的調酒，請你推薦我一杯調酒。同時也請你跟我說明夢酒館在地方所做的貢獻，還有夢酒館登過的國際媒體報導。」

將會使用這句話當作主要的問句給LLM



整合之後的回答是:

您好，非常感謝您的喜愛！如果您偏好清爽氣泡的調酒，我強烈推薦我們的「蜂蜜綠茶氣泡」。這款調酒以琴酒、百香果、芳香萬壽菊、茉莉綠茶、蜂蜜，再配上氣泡水調製而成，風味清新淡雅，帶有花香與果酸，入口輕盈、毫無負擔，就像是一場午後的湖畔約會，適合喜愛花果茶系風味、偏好輕酒感的朋友。它的金黃蜂蜜色和自然愜意的氛圍，也讓人聯想到午後陽光下的草皮野餐，非常適合想享受簡單、無負擔的時刻。

關於我們在地方的貢獻，夢酒館一向積極投入在地文化與創生行動。譬如，我們透過與金門大學合作，舉辦客座調酒與創業交流，促進在地青年與社區的連結。同時，我們更透過品牌策展，串連兩岸文化人，展現地方特色與文化深度。此外，夢酒館也曾經在2025年由金門縣政府深度報導中，展現其在地策展的能力與社群連結的努力，並參與多項青創與在地文化推動計畫。

在國際媒體部分，我們的故事也受到不少關注。夢酒館曾被 Reuters、AsiaOne、SCMP、Straits Times 等國際媒體報導，特別是我們的歷史主題調酒『Pick and Eat』，由於融合了金門戰地歷史元素，展現了品牌的文化深度。尚有 SCMP 製作短影片，介紹我們的空間與理念，讓國際讀者更了解我們的品牌故事。您可以參考以下連結了解詳細：
- [Reuters 報導：台灣前線酒吧將歷史融入調酒](https://www.reuters.com/world/asia-pacific/taiwan-bar-frontline-china-blends-history-into-cocktails-2024-05-26/)
- [SCMP 影片介紹夢酒館](https://www.youtube.com/watch?v=2S1TammmzhI)

如果您有進一步的喜好或想了解更多特別的調酒故事，也可以告訴我，我會依照您的口味來提供更貼近的建議！期待您在夢酒館享受到美好的時光！






# 7. 用 Gradio 打造 Web App

In [13]:
# 把上面三個步驟重新整合成一個 function 給 Gradio 使用

chat_history_local = []

def respond(message, chat_history_local):
    result = seperate_queries(message)
    main_query = result['main_query']
    queries_list = result['sub_queries']
    retrieved_answers = retrieve_answers(queries_list)
    response = integrate_answers(main_query, retrieved_answers)
    chat_history_local.append({"role": "user", "content": message})  # 使用者訊息
    chat_history_local.append({"role": "assistant", "content": response})  # 機器人回應
    return "", chat_history_local

#測試與示範
res = respond(user_input, chat_history_local)
res

('',
 [{'role': 'user',
   'content': '我喜歡清爽氣泡的調酒，請你推薦我一杯調酒。同時也請你跟我說明夢酒館在地方所做的貢獻，還有夢酒館登過的國際媒體報導。'},
  {'role': 'assistant',
   'content': '您好，非常感謝您的喜好！若您喜歡清爽氣泡的調酒，我會建議您試試我們「蜂蜜綠茶氣泡」。這款調酒是用琴酒、百香果、芳香萬壽菊、茉莉綠茶和蜂蜜，加上氣泡水調製而成，帶有清新、淡雅又不負擔的氣泡感。它的風味就像在湖畔的午後，帶著花香與果酸的親切甜感，口感輕盈，非常適合喜歡清爽氣泡調酒的您。（詳細可以參考我們的描述內容：蜂蜜綠茶風味明顯，有如初戀般的清新甜蜜。）\n\n關於夢酒館在地方的貢獻，我們不僅是一家調酒空間，更積極投入在地方創生與文化串連。例如，我們透過「金馬之夜」等文化活動，展現我們與在地團隊的合作與社群連結，串連兩岸文化人，並策劃多元的地方特色活動。此外，我們還與金門大學合作，邀請在地青年與學生共創，推動創業與文化交流，促進地方經濟與文化的活化（詳細資訊可參考：金門縣政府與青創活動的相關報導）。\n\n更值得一提的是，我們的故事與品牌理念也曾登上國際媒體如Reuters、AsiaOne、SCMP等。特別是Reuters曾報導我們以金門戰地歷史為靈感的創意調酒「Pick and Eat」，展現品牌的文化深度與國際傳播力。這些媒體的曝光不僅幫助我們推廣在地文化，也橋接了國際與在地的連結，讓更多人了解金門獨特的故事與我們的努力。（相關報導連結： [Reuters 報導](https://www.reuters.com/world/asia-pacific/taiwan-bar-frontline-with-china-blends-history-into-cocktails-2024-05-26/)、 [AsiaOne](https://www.asiaone.com/asia/taiwan-bar-frontline-china-blends-history-cocktails) 和 [SCMP影片](https://www.youtube.com/watch?v=2S1TammmzhI)）\n\n如果您想更了解我們的故事或特色，歡迎留下您的聯絡方式，或來夢酒館親自品味與體驗！我們很高興能用在

In [15]:
# 清空對話紀錄
chat_history = []
chat_history_local = []

with gr.Blocks() as demo:
    gr.Markdown("""
    # 夢酒館品牌大使 \n
    您好，我是夢酒館的品牌大使，可以問我任何有關夢酒館的問題，請問有什麼我可以為您服務的嗎？\n
    可以問我有關\n
    1. 調酒推薦 2. 我們的故事 3. 其他媒體報導
    """)

    chatbot = gr.Chatbot(type="messages") # 根據 Gemini 的說明，修改成新版支援的格式。

    msg = gr.Textbox(placeholder="請輸入你對夢酒館的好奇...")

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on public URL: https://0a9c55c2553160ad75.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0a9c55c2553160ad75.gradio.live


In [16]:
# 聊完天後，輸出聊天紀錄
index = 0
for chat in chat_history:
    index +=1
    print(f"{index}:\n使用者輸入:{chat[0]}\nRAG機器人:{chat[1]}\n\n")

1:
使用者輸入:你好啊
RAG機器人:您好！很高興在這裡與您相見。夢酒館不僅是一個品味精彩雞尾酒的場所，我們更是一個充滿故事與文化的空間。在這裡，您可以透過每一杯調酒探索金門的歷史與風土人情，像是「隨撿隨吃」帶出二戰時期的歷史記憶與福祉，以香菇湯意象展現地方客家的食材與溫暖情感，或是「毒鼠靈」讓您一窺金門那段特殊且堅韌的抗疫歷史，皆代表著我們對地方文化的尊重與創新。

我們的調酒不只是點飲料，更是一場文化的旅程。若您有特別的口味偏好，或想了解更多背景故事，歡迎告訴我們，我們會很樂意為您推薦最適合您的那一杯。

期待您能在夢酒館找到屬於自己的故事與回憶！如有任何需要或想進一步認識我們的故事，也請隨時聯絡我們，感謝您的光臨！


2:
使用者輸入:跟我介紹毒鼠靈
RAG機器人:您好，非常感謝您的問題。關於「毒鼠靈」這款調酒，我想跟您介紹一下它的故事與特色。

「毒鼠靈」這款調酒的名字來自於1950年代金門出現的鼠疫疫情，那時候為了防疫，軍方成立了「鼠疫防治處」，並運用了磷化鋅製成的毒鼠藥來控制老鼠，這種毒藥具有明顯的大蒜氣味。這段歷史雖然有些黑暗，但也是金門社會共同抗疫精神的象徵。

在我們的調酒中，使用了芋頭與芋泥的元素，重現那段用來引誘老鼠的餌劑之一的成分，以甜美的方式轉化這段歷史。味道上，甜膩濃郁、口感厚實，非常適合喜歡甜點或濃郁奶香的朋友，像是芋頭奶昔與甜點的結合，層層堆疊的香氣中包含著在地記憶。

這款調酒的視覺層次也很豐富，用紫灰色調呈現，像一碗富有裝飾的芋頭冰品，帶有一絲黑色歷史的甜美轉化。

如果您想了解煥發台灣地方歷史故事的調酒，或許會對它有興趣。此外，我們在金門也推動地方創生，希望透過這些故事與味道帶給每位旅客不一樣的體驗。

如果您有特別的口味偏好或喜好其他風味，也歡迎告訴我，我可以更針對您的喜好來推薦合適的調酒喔！


